# Setting Up

## Install Dependencies

In [1]:
!pip install -U --no-cache-dir tensorflow==2.3.0
!pip install -U numpy==1.18.5
!pip install -U Pillow==7.2.0
!pip install -U opencv-python
!pip install -U datature-hub==0.2.0


You should consider upgrading via the 'C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datature-hub 0.2.0 requires Pillow~=8.2, but you have pillow 7.2.0 which is incompatible.
You should consider upgrading via the 'C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.



  Using cached Pillow-7.2.0-cp38-cp38-win_amd64.whl (2.1 MB)
  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0
  Using cached opencv_python-4.5.5.64-cp36-abi3-win_amd64.whl (35.4 MB)
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.5.1.48
    Uninstalling opencv-python-4.5.1.48:
      Successfully uninstalled opencv-python-4.5.1.48


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datature-hub 0.2.0 requires opencv-python==4.5.1.48, but you have opencv-python 4.5.5.64 which is incompatible.
datature-hub 0.2.0 requires Pillow~=8.2, but you have pillow 7.2.0 which is incompatible.
You should consider upgrading via the 'C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


  Using cached Pillow-8.4.0-cp38-cp38-win_amd64.whl (3.2 MB)
  Using cached opencv_python-4.5.1.48-cp38-cp38-win_amd64.whl (34.9 MB)
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.2.0
    Uninstalling Pillow-7.2.0:
      Successfully uninstalled Pillow-7.2.0
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.5.5.64
    Uninstalling opencv-python-4.5.5.64:
      Successfully uninstalled opencv-python-4.5.5.64


You should consider upgrading via the 'C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


## Imports

In [2]:
import os
import time
import glob
import argparse
import numpy as np
import tensorflow as tf
import cv2
from datature_hub.hub import HubModel
from datature_hub.utils.visualize import visualize_bbox
from PIL import Image

## Comment out next line to use GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
## Comment out to set verbose to true
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Prediction Preparation

## Set all necessary variables

In [3]:
input_folder = "./input" # Path to folder that contains input images
output_folder = "./output" # Path to folder to store predicted images
threshold = 0.2 # Prediction confidence threshold

project_secret = "76d97105923491bfa13c84d74eb5457b3b04dceda19ca009d7af111bd7d05344"#"<YOUR_PROJECT_SECRET_KEY>"
model_key = "f2324a0064025c01da8fe3482177a83a"#"<YOUR_MODEL_KEY>"


## Initialize Hub

In [4]:
hub_model = HubModel(
    project_secret=project_secret,
    model_key=model_key,
  )

## Load the model and label map

In [5]:
trained_model = hub_model.load_tf_model()
category_index = hub_model.load_label_map()

## Run Prediction on Each Image

In [6]:
all_detections = []
all_images = glob.glob(os.path.join(input_folder, "*"))
for each_image in all_images:
  print("Predicting for {}...".format(each_image))
  input_tensor = hub_model.load_image_with_model_dimensions(each_image)
  detections_output = trained_model(input_tensor)
  all_detections.append(detections_output)

Predicting for ./input\boat161.png...


ResourceExhaustedError:  OOM when allocating tensor with shape[346800,1088] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node StatefulPartitionedCall/MultiLevelMatMulCropAndResize/MultiLevelRoIAlign/GatherV2_1}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_restored_function_body_106372]

Function call stack:
restored_function_body


## Save Image Visualizations

In [ ]:
for each_image, each_detection in zip(all_images, all_detections):
  visualized_image = visualize_bbox(each_image, each_detection, category_index, threshold)
  filename = os.path.basename(each_image)
  image_predict = Image.fromarray(visualized_image)
  image_predict.save(os.path.join(output_folder, filename))

  print(
    "Saving predicted images to {}...".format(
      os.path.join(output_folder, filename)
    )
  )